In [1]:
# Necessary libraries
import os
import sys
import csv
import math
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

from pprint import pprint
import requests
import json

In [2]:
# Ach so...

# Import History Bulk data for Houston from OpenWeatherMap
city_list_path = os.path.join("HistoricalWeatherByCity", "4850674b7d8d779ae028900dd58b078e.json")

# create an empty DataFrame to store Houston Data
houston_df = pd.DataFrame()
day_date = []
weather = []

# read in the file
with open(city_list_path, encoding="utf8") as fp:     # must use UTF8 or you'll be in trouble
    city_list_data = fp.read()
    city_list_json = json.loads(city_list_data)       # seems like reading from string is faster than reading directly from file handle
    #pprint(city_list_json)
    
# Loop through the history data of Houston
for day in city_list_json:
    #pprint(day)
    day_date.append(day["dt_iso"].split(" ")[0])      # we knocked off the time of the day, keeping only the day
    weather.append(day["weather"][0]["main"])
        
# Add date and weather to dataframe
houston_df["Date"] = day_date
houston_df["Weather"] = weather
houston_df

,Date,Weather
0,2015-06-12,Clear
1,2015-06-12,Clouds
2,2015-06-13,Clouds
3,2015-06-13,Clouds
4,2015-06-14,Rain
5,2015-06-14,Rain
6,2015-06-15,Clouds
7,2015-06-19,Clouds
8,2015-06-20,Clouds
9,2015-06-22,Clouds


In [3]:
# Since each day has 24 entries at this moment (because they're combined from different times of the day),
# We combine the days by getting the "main" weather of the day
houston_gby = houston_df.groupby("Date")
houston_df_new = houston_gby.describe()

# Get rid of the top level column "Weather"
houston_df_new.columns = houston_df_new.columns.droplevel()

# Reduce the index "Day" to a regular column
houston_df_new = houston_df_new.drop(columns = ["count", "unique", "freq"])
houston_df_new = houston_df_new.rename(columns = {"top": "Weather"})
houston_df_new = houston_df_new.reset_index(level = "Date")
houston_df_new


,Date,Weather
0,2015-06-12,Clouds
1,2015-06-13,Clouds
2,2015-06-14,Rain
3,2015-06-15,Clouds
4,2015-06-19,Clouds
5,2015-06-20,Clouds
6,2015-06-22,Clouds
7,2015-06-24,Clear
8,2015-06-25,Clear
9,2015-06-26,Rain


In [6]:
# Export csv file to be read from code
weather_day_houston = os.path.join("HistoricalWeatherByCity", "weather_day_houston.csv")
houston_df_new.to_csv(weather_day_houston)

In [5]:
# # Trying to pull historical weather data of Houston
# api_key = "5e49b0c850789c4d3deba0e75377ec98"
# url = "http://api.openweathermap.org/data/2.5/history/city?"
# city = "Houston, US"
# units = "imperial"
# start = "1420070400"
# end = "1451606400"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city +"&units=" + units + "&type=hour" + "&start=" + start + "&end=" + end

# # Get data
# weather_houston = requests.get(query_url).json()
# pprint(weather_houston)